In [25]:
import pandas as pd
import random 
import json

experiment_folder = "fifth_experiment"

tech_file_path = f'data/experiments/{experiment_folder}/408_tech_sentences_with_prompts.csv'
house_file_path = f'data/experiments/{experiment_folder}/208_house_sentences_with_prompts.csv'

tech_sentences_df = pd.read_csv(tech_file_path)
house_sentences_df = pd.read_csv(house_file_path)

with open(f"data/experiments/{experiment_folder}/442_house_rewrites.json", 'r') as f:
    house_rewrites = json.load(f)
    
with open(f"data/experiments/{experiment_folder}/846_tech_rewrites.json", 'r') as f:
    tech_rewrites = json.load(f)


tech_sentences_df

,occupation_name,skills,knowledge,prompt,results,rephrase_prompts
0,system architect,['create data models'],[],"Take this sentence from a Job Posting.\n\n""* P...",* Passionate about @@creating data models@@,NaN
1,data quality specialist,[],['N1QL'],"Take this sentence from a Job Posting.\n\n""• B...",• Basic ##N1QL##.,NaN
2,chief security officer,['optimise choice of solution'],[],"Take this sentence from a Job Posting.\n\n""@@C...",@@Optimise the choice of solution@@ to transfo...,Job-related knowledge is surrounded by ## symb...
3,user interface designer,['develop creative ideas'],['DevOps'],"Take this sentence from a Job Posting.\n\n""Und...",Understand ##DevOps## from both a managerial/s...,NaN
4,embedded system designer,['develop creative ideas'],[],"Take this sentence from a Job Posting.\n\n""@@B...",@@Develop creative ideas@@ to meet endpoint pr...,NaN
...,...,...,...,...,...,...
403,digital forensics expert,[],"[' security standards', 'John The Ripper (pene...","Take this sentence from a Job Posting.\n\n""Par...",Participates in ##security standards## and ##J...,NaN
404,security engineer,[],['internet governance'],"Take this sentence from a Job Posting.\n\n""Fam...",Familiarity with the ##internet governance## i...,NaN
405,security manager,['define technology strategy'],[],"Take this sentence from a Job Posting.\n\n""As ...",As an experienced member of our <ORGANIZATION>...,NaN
406,chief security officer,[],['internal risk management policy'],"Take this sentence from a Job Posting.\n\n""##C...",##Internal risk management policy##,NaN


In [26]:
def convert_to_bio_tagged_sentences(sentence):
    words = sentence.replace('##', ' ## ').replace('@@', ' @@ ').split()
    tagged_words = []
    
    skill_tag = 'O'  
    knowledge_tag = 'O'  

    for word in words:
        if word == '@@':
            if skill_tag == 'O':
                skill_tag = 'B-Skill'  
            else:
                skill_tag = 'O'  
            continue  
        
        if word == '##':
            if knowledge_tag == 'O':
                knowledge_tag = 'B-Knowledge'  
            else:
                knowledge_tag = 'O'  
            continue  
        
        tagged_words.append((word, skill_tag, knowledge_tag))

        if skill_tag == 'B-Skill':
            skill_tag = 'I-Skill'
        if knowledge_tag == 'B-Knowledge':
            knowledge_tag = 'I-Knowledge'
    
    return tagged_words

In [27]:
def split_sentences_to_bio(sentences_to_split):
    splitted_sentences = []

    for sentence_to_split in sentences_to_split:
        splitted_sentences.append(convert_to_bio_tagged_sentences(sentence_to_split))
        
    return splitted_sentences

In [28]:
def append_sentences_to_conll(filename, sentences_to_append):
    with open(filename, 'a') as file:
        for sentence in sentences_to_append:
            file.write("\n")  
            for word, skill_tag, knowledge_tag in sentence:
                file.write(f"{word}\t{skill_tag}\t{knowledge_tag}\n")


In [29]:
bio_splitted_sentences = split_sentences_to_bio(tech_sentences_df['results'])
append_sentences_to_conll(f"data/experiments/{experiment_folder}/enriched_skillspan/skillspan_tech_train.conll", bio_splitted_sentences)

In [30]:
bio_splitted_sentences = split_sentences_to_bio(house_sentences_df['results'])
append_sentences_to_conll(f"data/experiments/{experiment_folder}/enriched_skillspan/skillspan_house_train.conll", bio_splitted_sentences)

In [31]:
bio_splitted_sentences = split_sentences_to_bio(tech_rewrites)
append_sentences_to_conll(f"data/experiments/{experiment_folder}/enriched_skillspan/skillspan_tech_train.conll", bio_splitted_sentences)

In [32]:
bio_splitted_sentences = split_sentences_to_bio(house_rewrites)
append_sentences_to_conll(f"data/experiments/{experiment_folder}/enriched_skillspan/skillspan_house_train.conll", bio_splitted_sentences)